In [ ]:
# All imports here
import numpy as np
import pandas as pd
import keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
import copy

In [ ]:
# Define the paths here
one_million_path = "/content/drive/MyDrive/Sudoku_Solver/data/1Million.csv"

In [ ]:
# Read in pandas
one_million_dataset = pd.read_csv(one_million_path)

In [ ]:
one_million_dataset.columns

Index(['quizzes', 'solutions'], dtype='object')

In [ ]:
# Helper method Split the data set into Train and Test Splits
def train_test_split_data(df, test_ratio=0.2): 

    questions = []
    solutions = []

    # Process questions and solutions
    for idx, row in df.iterrows():
      # Pre process question
      question = row['quizzes']
      question = (np.array(list(map(int,list(question)))).reshape((9,9,1))/9) - 0.5
      # print(question.shape)
      questions.append(question)

      # Pre process solutions
      solution = row['solutions']
      solution = (np.array(list(map(int,list(solution)))).reshape(81, 1)) - 1
      # print("Solution", solution.shape)
      solutions.append(solution)

    x_train, x_test, y_train, y_test = train_test_split(np.array(questions), np.array(solutions), test_size=test_ratio, random_state=42)

    del(questions)
    del(solutions)
    
    return x_train, x_test, y_train, y_test

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split_data(one_million_dataset)

In [ ]:
print(len(X_train), len(Y_train), len(x_test), len(y_test))

800000 800000 200000 200000


In [ ]:
X_train.shape

(800000, 9, 9, 1)

In [ ]:
# Build the model
def buildModel():

  model = Sequential()

  # Layer 1
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
  model.add(BatchNormalization())

  # Layer 2
  model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
  model.add(BatchNormalization())

  # Layer 3
  model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

  model.add(Flatten())
  model.add(Dense(81*9))
  model.add(Reshape((-1, 9)))
  model.add(Activation('softmax'))

  adam = Adam(learning_rate=.001)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

  return model

In [ ]:
# Initialize the model
model1 = buildModel()
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 64)          640       
                                                                 
 batch_normalization (BatchN  (None, 9, 9, 64)         256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_1 (Batc  (None, 9, 9, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 128)         8320      
                                                                 
 flatten (Flatten)           (None, 10368)             0

In [ ]:
# Fit the model on train data
model1.fit(X_train, Y_train, batch_size=128, epochs=10)

Epoch 1/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2926 - accuracy: 0.8700
Epoch 2/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2920 - accuracy: 0.8704
Epoch 3/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2915 - accuracy: 0.8708
Epoch 4/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2909 - accuracy: 0.8710
Epoch 5/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2904 - accuracy: 0.8713
Epoch 6/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2899 - accuracy: 0.8716
Epoch 7/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2894 - accuracy: 0.8719
Epoch 8/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2890 - accuracy: 0.8721
Epoch 9/10
6250/6250 [==============================] - 31s 5ms/step - loss: 0.2885 - accuracy: 0.8724
Epoch 10/10
6250/6250 [==============================] - 31s 5ms/step - l

In [ ]:
# Save the model
model1.save("/content/drive/MyDrive/Sudoku_Solver/data/model_{1M}_{32E}_{87A}.h5")